In [ ]:
%pip uninstall -y google-generativeai google-ai-generativelanguage
%pip install langchain
%pip install -qU "langchain[google-genai]"
%pip install langchain_community
%pip install -U langchain-tavily
%pip install -U langchain-huggingface
%pip install -qU langchain-google-genai
%pip install faiss-cpu
%pip install pypdf
import os
import getpass
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
load_dotenv()
# .env should also have TAVILY_API_KEY
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "default"
if not os.environ.get("TAVILY_API_KEY"):
  os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter API key for Tavily: ")
if not os.environ.get("HF_TOKEN"):
  os.environ["HF_TOKEN"] = getpass.getpass("Enter API key for Hugging Face: ")
if not os.environ.get("GEMINI_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")
else:
  os.environ["GOOGLE_API_KEY"] = os.environ.get("GEMINI_API_KEY")
model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

Found existing installation: google-ai-generativelanguage 0.6.18
Uninstalling google-ai-generativelanguage-0.6.18:
  Successfully uninstalled google-ai-generativelanguage-0.6.18


In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="translate the sentence into 5 different languages."),
    HumanMessage(content="Hello junior, where is your father?"),
]

model.invoke(messages)

AIMessage(content='Here\'s the sentence "Hello junior, where is your father?" translated into 5 different languages:\n\n1.  **Spanish:** ¡Hola joven, ¿dónde está tu padre?\n2.  **French:** Bonjour jeune homme, où est ton père ?\n3.  **German:** Hallo Kleiner, wo ist dein Vater?\n4.  **Italian:** Ciao ragazzino, dov\'è tuo padre?\n5.  **Portuguese:** Olá garoto, onde está seu pai?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--401a717d-c427-4c24-93ea-8560b1f0081e-0', usage_metadata={'input_tokens': 18, 'output_tokens': 745, 'total_tokens': 763, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 638}})

In [ ]:
for token in model.stream(messages):
    print(token.content, end="#")

Here are 5 translations of "Hello junior, where is your father?":

1.  **Spanish:** Hola junior, ¿dónde está tu padre?
2.  **French:** Bonjour junior, où est ton père ?#
3.  **German:** Hallo Junior, wo ist dein Vater?
4.  **Italian:** Ciao junior, dov'è tuo padre?
5.  **Portuguese:** Olá júnior, onde está seu pai?#

In [ ]:
def get_response(Query: str) -> str:
  structured_llm = set_response(Query)
  return structured_llm.invoke(Query)
def set_response(Query: str) -> str:
  if "joke" in Query.lower():
    structured_llm = model.with_structured_output(Joke)
  else:
    structured_llm = model.with_structured_output(ConversationalResponse)
  return structured_llm

In [ ]:
from typing import Optional, Union
from pydantic import BaseModel, Field

# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(default=None, description="How funny the joke is, from 1 to 10")

class ConversationalResponse(BaseModel):
    """Respond in a conversational manner. Be kind and helpful."""
    response: str = Field(description="A conversational response to the user's query")

get_response("Tell me something about cats")


ConversationalResponse(response="Cats are fascinating creatures! They're known for their independence, agility, and their adorable purrs. Did you know that domestic cats have been living alongside humans for thousands of years? They're also incredibly diverse, with many different breeds, each with its own unique characteristics. Is there anything specific you'd like to know about them?")

In [ ]:
from typing_extensions import Annotated, TypedDict

# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""
    setup: Annotated[str, ..., "The setup of the joke"]
    # Alternatively, we could have specified setup as:
    # setup: str                    # no default, no description
    # setup: Annotated[str, ...]    # no default, no description
    # setup: Annotated[str, "foo"]  # default, no description
    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]

class ConversationalResponse(TypedDict):
    """Respond in a conversational manner. Be kind and helpful."""
    response: Annotated[str, ..., "A conversational response to the user's query"]

get_response("Tell me a joke about dogs with rating ofcourse")

{'rating': 7.0,
 'punchline': 'Because they have two left feet!',
 'setup': "Why don't dogs make good dancers?"}

In [ ]:
Joke = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}

ConversationalResponse = {
    "title": "conversational_response",
    "description": "Respond in a conversational manner. Be kind and helpful.",
    "type": "object",
    "properties": {
        "response": {
            "type": "string",
            "description": "A conversational response to the user's query",
        }
    },
    "required": ["response"],
}

get_response("Tell me about bermuda triangle!")

{'response': "The Bermuda Triangle, also known as the Devil's Triangle, is a loosely defined region in the western part of the North Atlantic Ocean where a number of aircraft and ships are said to have disappeared under mysterious circumstances. It's a fascinating subject with lots of theories! Some people attribute the disappearances to paranormal activity or even extraterrestrial involvement, while others point to environmental factors like hurricanes, rogue waves, or the compass variations in the area. It's a mystery that continues to capture the imagination!"}

In [ ]:
Query = "joke of triangle? of rating below 4 but not None"
structured_llm = set_response(Query)
for chunk in structured_llm.stream(Query):
    print(chunk)

{'rating': 3.0, 'punchline': 'Because it had three strong sides!', 'setup': 'Why was the triangle the strongest shape?'}


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?").

Here are some examples of jokes:

example_user: Tell me a joke about planes
example_assistant: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}

example_user: Tell me another joke about planes
example_assistant: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}

example_user: Now about caterpillars
example_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke("what's something funny about woodpeckers")# method can be json and include_raw can be true

{'rating': 8.0,
 'punchline': "Woodpecker you please open the door, I've been knocking for ages!",
 'setup': 'Woodpecker'}

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

examples = [
    HumanMessage("Tell me a joke about planes", name="your_name"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Why don't planes ever get tired?",
                    "punchline": "Because they have rest wings!",
                    "rating": 2,
                },
                "id": "1",
            }
        ],
    ),
    # Most tool-calling models expect a ToolMessage(s) to follow an AIMessage with tool calls.
    ToolMessage("", tool_call_id="1"),
    # Some models also expect an AIMessage to follow any ToolMessages,
    # so you may need to add an AIMessage here.
    HumanMessage("Tell me another joke about planes", name="your_name"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Cargo",
                    "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!",
                    "rating": 10,
                },
                "id": "2",
            }
        ],
    ),
    ToolMessage("", tool_call_id="2"),
    HumanMessage("Now about caterpillars", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Caterpillar",
                    "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!",
                    "rating": 5,
                },
                "id": "3",
            }
        ],
    ),
    ToolMessage("", tool_call_id="3"),
]
system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") \
and the final punchline (the response to "<setup> who?")."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("placeholder", "{examples}"), ("human", "{input}")]
)
few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke({"input": "crocodiles", "examples": examples})

{'rating': 7.0,
 'punchline': "Crocodile you later, I'm just here to make a splash!",
 'setup': 'Crocodile'}

In [ ]:
from typing import List
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(
        ..., description="The height of the person expressed in meters."
    )


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]


# Set up a parser
parser = PydanticOutputParser(pydantic_object=People)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

query = "Anna is 23 years old and she is 6 feet tall"

print(prompt.invoke({"query": query}).to_string())

System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Person": {"description": "Information about a person.", "properties": {"name": {"description": "The name of the person", "title": "Name", "type": "string"}, "height_in_meters": {"description": "The height of the person expressed in meters.", "title": "Height In Meters", "type": "number"}}, "required": ["name", "height_in_meters"], "title": "Person", "type": "object"}}, "description": "Identifying information about all people in a text.", "properties": {"people": {"items"

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
parser = StrOutputParser()
chain = prompt | model | parser

async for chunk in chain.astream({"topic": "parrot"}):
    print(chunk, end="|", flush=True)

Here are a few parrot jokes for you:

1.  A man walks into a pet shop and sees a parrot with a sign that says, "Talks in two languages."
    He asks the parrot, "Can you say something in| Spanish?"
    The parrot replies, "Hola!"
    The man says, "Great! Now, what about English?"
    The parrot replies, "Hola!"

2.  Why did the pirate get a parrot?
|    Because he liked to swear!

3.  A man buys a parrot that swears like a sailor. He tries everything to stop it: covering its cage, putting it in a dark room. Finally, he throws it in the freezer|.
    After a few minutes, he takes it out. The parrot shivers and says, "I'm very sorry for my language. Could you please tell me what the chicken did?"

4.  What do you call a| parrot that flies away?
    A *Polly-gone*!

5.  A woman buys a talking parrot. She brings it home and introduces it to her husband. The parrot immediately says, "Wow, you're a| lot uglier than your husband described!"|

In [ ]:
from langchain_core.output_parsers import JsonOutputParser


async def _extract_country_names_streaming(input_stream):
    """A function that operates on input streams."""
    country_names_so_far = set()

    async for input in input_stream:
        if not isinstance(input, dict):
            continue

        if "countries" not in input:
            continue

        countries = input["countries"]

        if not isinstance(countries, list):
            continue

        for country in countries:
            name = country.get("name")
            if not name:
                continue
            if name not in country_names_so_far:
                yield name
                country_names_so_far.add(name)


chain = model | JsonOutputParser() | _extract_country_names_streaming

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    print(text, end="|", flush=True)

France|Spain|Japan|

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "harrison likes spicy food"],
    embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
)
retriever = vectorstore.as_retriever()

chunks = [chunk for chunk in retriever.stream("where did harrison work?")]
chunks

[[Document(id='66600e88-6c69-476d-a6c5-ec255ab655fc', metadata={}, page_content='harrison worked at kensho'),
  Document(id='cd0e01dc-b163-4735-8af8-d32cb515de56', metadata={}, page_content='harrison likes spicy food')]]

In [ ]:
chain = model.with_config({"run_name": "model"}) | JsonOutputParser().with_config(
    {"run_name": "my_parser"}
)

max_events = 0
async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
    include_names=["my_parser"], #include_types=["chat_model"],or include_tags=["my_chain"],
):
    print(event)
    max_events += 1
    if max_events > 10:
        # Truncate output
        print("...")
        break

{'event': 'on_parser_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'my_parser', 'tags': ['seq:step:2'], 'run_id': '3dcf2c07-641e-412e-b3a8-809cc66eaf58', 'metadata': {}, 'parent_ids': ['8ea6f1ec-e761-4bc3-9d79-7e11f85febeb']}
{'event': 'on_parser_stream', 'run_id': '3dcf2c07-641e-412e-b3a8-809cc66eaf58', 'name': 'my_parser', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': {'countries': [{'name': 'France', 'population': 68042000}, {'name': 'Spain'}]}}, 'parent_ids': ['8ea6f1ec-e761-4bc3-9d79-7e11f85febeb']}
{'event': 'on_parser_stream', 'run_id': '3dcf2c07-641e-412e-b3a8-809cc66eaf58', 'name': 'my_parser', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': {'countries': [{'name': 'France', 'population': 68042000}, {'name': 'Spain', 'population': 476160

In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.tools import tool


def reverse_word(word: str):
    return word[::-1]

reverse_word = RunnableLambda(reverse_word)

@tool
def bad_tool(word: str):
    """Custom tool that doesn't propagate callbacks."""
    return reverse_word.invoke(word)

async for event in bad_tool.astream_events("hello"):
    print(event)

{'event': 'on_tool_start', 'data': {'input': 'hello'}, 'name': 'bad_tool', 'tags': [], 'run_id': '73902902-92a2-4b92-a4a9-f4e22b4313d9', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': 'hello'}, 'name': 'reverse_word', 'tags': [], 'run_id': '01c098f5-f330-4990-9a0d-b89e30cfcfb2', 'metadata': {}, 'parent_ids': ['73902902-92a2-4b92-a4a9-f4e22b4313d9']}
{'event': 'on_chain_end', 'data': {'output': 'olleh', 'input': 'hello'}, 'run_id': '01c098f5-f330-4990-9a0d-b89e30cfcfb2', 'name': 'reverse_word', 'tags': [], 'metadata': {}, 'parent_ids': ['73902902-92a2-4b92-a4a9-f4e22b4313d9']}
{'event': 'on_tool_end', 'data': {'output': 'olleh'}, 'run_id': '73902902-92a2-4b92-a4a9-f4e22b4313d9', 'name': 'bad_tool', 'tags': [], 'metadata': {}, 'parent_ids': []}


In [ ]:
@tool
def correct_tool(word: str, callbacks):
    """A tool that correctly propagates callbacks."""
    return reverse_word.invoke(word, {"callbacks": callbacks})


async for event in correct_tool.astream_events("hello"):
    print(event)

{'event': 'on_tool_start', 'data': {'input': 'hello'}, 'name': 'correct_tool', 'tags': [], 'run_id': 'fc8fbdf0-4ad4-4eaf-b23b-3ef46b7b4fc9', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': 'hello'}, 'name': 'reverse_word', 'tags': [], 'run_id': 'e68a0f4f-35b3-44e7-9600-a6f019ddf9ff', 'metadata': {}, 'parent_ids': ['fc8fbdf0-4ad4-4eaf-b23b-3ef46b7b4fc9']}
{'event': 'on_chain_end', 'data': {'output': 'olleh', 'input': 'hello'}, 'run_id': 'e68a0f4f-35b3-44e7-9600-a6f019ddf9ff', 'name': 'reverse_word', 'tags': [], 'metadata': {}, 'parent_ids': ['fc8fbdf0-4ad4-4eaf-b23b-3ef46b7b4fc9']}
{'event': 'on_tool_end', 'data': {'output': 'olleh'}, 'run_id': 'fc8fbdf0-4ad4-4eaf-b23b-3ef46b7b4fc9', 'name': 'correct_tool', 'tags': [], 'metadata': {}, 'parent_ids': []}


In [ ]:
from langchain_core.runnables import RunnableLambda

async def reverse_and_double(word: str):
    return await reverse_word.ainvoke(word) * 2

reverse_and_double = RunnableLambda(reverse_and_double)

await reverse_and_double.ainvoke("1234")

async for event in reverse_and_double.astream_events("1234"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_and_double', 'tags': [], 'run_id': 'eadb2cb0-0a09-40bf-8029-4ea703212d09', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_word', 'tags': [], 'run_id': '2f7bb1b4-f191-4fea-99a6-29d80fe64349', 'metadata': {}, 'parent_ids': ['eadb2cb0-0a09-40bf-8029-4ea703212d09']}
{'event': 'on_chain_end', 'data': {'output': '4321', 'input': '1234'}, 'run_id': '2f7bb1b4-f191-4fea-99a6-29d80fe64349', 'name': 'reverse_word', 'tags': [], 'metadata': {}, 'parent_ids': ['eadb2cb0-0a09-40bf-8029-4ea703212d09']}
{'event': 'on_chain_stream', 'run_id': 'eadb2cb0-0a09-40bf-8029-4ea703212d09', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'data': {'chunk': '43214321'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': '43214321'}, 'run_id': 'eadb2cb0-0a09-40bf-8029-4ea703212d09', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'parent_ids': []}


In [ ]:
from langchain_core.runnables import chain

@chain
async def reverse_and_double(word: str):
    return await reverse_word.ainvoke(word) * 2

await reverse_and_double.ainvoke("1234")

async for event in reverse_and_double.astream_events("1234"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_and_double', 'tags': [], 'run_id': '4b1cd715-f5ba-44d8-a866-c68b265c3e71', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_word', 'tags': [], 'run_id': '4cd2e0e6-81bd-45ba-969c-96c69779cb3f', 'metadata': {}, 'parent_ids': ['4b1cd715-f5ba-44d8-a866-c68b265c3e71']}
{'event': 'on_chain_end', 'data': {'output': '4321', 'input': '1234'}, 'run_id': '4cd2e0e6-81bd-45ba-969c-96c69779cb3f', 'name': 'reverse_word', 'tags': [], 'metadata': {}, 'parent_ids': ['4b1cd715-f5ba-44d8-a866-c68b265c3e71']}
{'event': 'on_chain_stream', 'run_id': '4b1cd715-f5ba-44d8-a866-c68b265c3e71', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'data': {'chunk': '43214321'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': '43214321'}, 'run_id': '4b1cd715-f5ba-44d8-a866-c68b265c3e71', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'parent_ids': []}


In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_tavily import TavilySearch

tools = [TavilySearch(max_results=5, topic="general")]
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(model, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke(
    {"input": "Who directed the 2023 film Oppenheimer and what is his age in days?"}
)

{'input': 'Who directed the 2023 film Oppenheimer and what is his age in days?',
 'output': 'Christopher Nolan directed the 2023 film *Oppenheimer*. He was born on July 30, 1970, which makes him 19,478 days old as of November 27, 2023.'}

In [ ]:
from langchain.globals import set_verbose

set_verbose(True)
agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke(
    {"input": "Who directed the 2023 film Oppenheimer and what is their age in days?"}
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': 'Who directed the 2023 film Oppenheimer'}`


{'query': 'Who directed the 2023 film Oppenheimer', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://letterboxd.com/film/oppenheimer-2023/', 'title': 'Oppenheimer (2023) directed by Christopher Nolan - Letterboxd', 'content': "Cillian Murphy Emily Blunt Matt Damon Robert Downey Jr. Florence Pugh Josh Hartnett Casey Affleck Rami Malek Kenneth Branagh Benny Safdie Jason Clarke Dylan Arnold Tom Conti James D'Arcy David Dastmalchian Dane DeHaan Alden Ehrenreich Tony Goldwyn Jefferson Hall David Krumholtz Matthew Modine Scott Grimes Kurt Koehler John Gowans Macon Blair Harry Groener Gregory Jbara Ted King Tim DeKay Show All…  Steven Houska Petrie Willink Matthias Schweighöfer Alex Wolff Josh Zuckerman Rory Keane Michael Angarano Emma Dumont Sadie Stratton Britt Kyle Guy Burnet Tom Jenkins Louise Lombard Michael Andrew Baker Je

{'input': 'Who directed the 2023 film Oppenheimer and what is their age in days?',
 'output': 'Christopher Nolan directed the 2023 film Oppenheimer. As of November 19, 2023, he is 19,469 days old.'}

In [ ]:
from langchain.globals import set_debug

set_debug(True)
set_verbose(False)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke(
    {"input": "Who directed the 2023 film Oppenheimer and what is their age in days?"}
)

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Who directed the 2023 film Oppenheimer and what is their age in days?"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiti

{'input': 'Who directed the 2023 film Oppenheimer and what is their age in days?',
 'output': 'Christopher Nolan directed the 2023 film *Oppenheimer*. As of October 27, 2024, Christopher Nolan, who was born on July 30, 1970, is 19,813 days old.'}